In [40]:
#pip install "git+https://github.com/optimizacion-2-2021-1-gh-classroom/practica-2-segunda-parte-caroacostatovany.git#egg=mex&subdirectory=src"

In [38]:
#pip uninstall mex -y

In [2]:
import numpy as np
import timeit

In [3]:
import scipy.io as sio

In [4]:
mat = sio.loadmat('../data/AFIRO.mat')
#mat = sio.loadmat('../data/80BAU3B.mat')

In [5]:
A=mat['A']
A.shape

(27, 51)

In [6]:
b=mat['b']
len(b)

27

In [7]:
c=mat['c']
len(c)

51

In [8]:
lbounds=mat['lbounds']
ubounds=mat['ubounds']
print(len(lbounds))
print(len(ubounds))

51
51


In [23]:
def generates_matrix(A, b, blow, bupp, c, compr = True):
    """
    Genera la matriz bajo el supuesto de que se tiene ya capturado los datos de entrada. Genera el problema del tipo:
    `
    Al tipo
    Ax <= b
    Concantenando renglones de A y juntando los limites de las restricciones (multiplicando por -1 donde sea necesario)
    """
    
    if compr:
        A = A.todense()
        A = np.array(A)
    
    n_restr, n_vars = A.shape

    # Concatenar A con matrices identidades de variables sinteticas de forma rbind
    identidad_vars_sint = np.eye(n_vars, n_vars)
    c = c.T
    A_new = np.concatenate((A, identidad_vars_sint, -identidad_vars_sint, c), axis=0)

    # Concatenar variables coeficientes sinteticas
    A_sint = np.eye(n_restr + 2*n_vars + 1, n_restr + 2*n_vars + 1)
    A_sint[0:n_restr,:] = A_sint[0:n_restr,:]*0
    
    # Concatenar A_new con A_sint de formca rbind
    A_new = np.c_[A_new, A_sint]
    
    # Juntar cotas de restricciones
    b_new = np.concatenate((b, bupp, -blow, np.array([[0]])), axis=0)
    
    # Concatenar A_new, b_new de forma cbind
    A_new = np.c_[A_new, b_new]
    
    return A_new

# Prueba de paquetería `mex`

In [24]:
from mex.simplex.simplex_networks import create_matrix, pivots_col, pivots_row, \
find_negative_col, find_negative_row, find_pivot_col, find_pivot_row, pivot

from mex.simplex.problem_definition import add_cons, \
constrain, add_obj, obj, maxz, minz

In [25]:
matrix_min_approx = create_matrix(2,7)
matrix_min_approx

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [26]:
constrain(matrix_min_approx,'1,1,E,6')
constrain(matrix_min_approx,'-1,2,E,6')
constrain(matrix_min_approx,'4,3,E,6')
constrain(matrix_min_approx,'1,0,G,0')
constrain(matrix_min_approx,'0,1,G,0')
constrain(matrix_min_approx,'1,0,L,18')
constrain(matrix_min_approx,'0,1,L,18')
obj(matrix_min_approx,'-1,-3,0')
matrix_min_approx

array([[ 1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  6.],
       [-1.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  6.],
       [ 4.,  3.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  6.],
       [-1., -0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0., -0.],
       [-0., -1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0., -0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0., 18.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0., 18.],
       [ 1.,  3.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.]])

In [27]:
A_prueba = np.array([[1,1], [-1,2], [4,3]])
b_prueba = np.array([[6,6,6]]).T
c_prueba = np.array([[1,3]]).T
blow_prueba = np.array([[0,0]]).T
bupp_prueba = np.array([[18,18]]).T

In [28]:
generates_matrix(A_prueba, b_prueba, blow_prueba, bupp_prueba, c_prueba, False)

array([[ 1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  6.],
       [-1.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  6.],
       [ 4.,  3.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  6.],
       [ 1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0., 18.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0., 18.],
       [-1., -0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [-0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 1.,  3.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.]])

### Prueba del paquete `mex`

In [29]:
mat = sio.loadmat('../data/AFIRO.mat')
#mat = sio.loadmat('../data/80BAU3B.mat')

In [30]:
A = mat['A']
b = mat['b']
lbounds = mat['lbounds']
ubounds = mat['ubounds']
c = mat['c']

In [31]:
print(A.shape)

(27, 51)


In [32]:
tableau = generates_matrix(A, b, lbounds, ubounds, c)

Comenzó a correr a las $3:48:00$

In [33]:
start = timeit.timeit()

problem_approx = minz(tableau)

end = timeit.timeit()
print(end - start)


TypeError: 'NoneType' object is not subscriptable

In [34]:
l = len(tableau[:, 0])
min(tableau[l-1, :-1])

-10.0

In [35]:
tableau.shape

(130, 182)